# Extracting training data from the ODC  <img align="right" src="../../Supplementary_data/dea_logo.jpg">

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the `DEA Sandbox` environment
* **Products used:** 
[ga_ls8c_nbart_gm_cyear_3](https://explorer.dea.ga.gov.au/products/ga_ls8c_nbart_gm_cyear_3),
[ga_ls_fc_pc_cyear_3](https://explorer.dea.ga.gov.au/products/ga_ls_fc_pc_cyear_3)

## Background

**Training data** is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy ([Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)).  A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034) 

When creating training labels, be sure to capture the **spectral variability** of the class, and to use imagery from the time period you want to classify (rather than relying on basemap composites). Another common problem with training data is **class imbalance**. This can occur when one of your classes is relatively rare and therefore the rare class will comprise a smaller proportion of the training set. When imbalanced data is used, it is common that the final classification will under-predict less abundant classes relative to their true proportion.

There are many platforms to use for gathering training labels, the best one to use depends on your application. GIS platforms are great for collection training data as they are highly flexible and mature platforms; [Geo-Wiki](https://www.geo-wiki.org/) and [Collect Earth Online](https://collect.earth) are two open-source websites that may also be useful depending on the reference data strategy employed. Alternatively, there are many pre-existing training datasets on the web that may be useful, e.g. [Radiant Earth](https://www.radiant.earth/) manages a growing number of reference datasets for use by anyone.


## Description
This notebook will extract training data (feature layers, in machine learning parlance) from the `open-data-cube` using labelled geometries within a geojson. The default example will use the crop/non-crop labels within the `'data/crop_training_WA.geojson'` file. This reference data was acquired and pre-processed from the USGS's Global Food Security Analysis Data portal [here](https://croplands.org/app/data/search?page=1&page_size=200) and [here](https://e4ftl01.cr.usgs.gov/MEASURES/GFSAD30VAL.001/2008.01.01/).

To do this, we rely on a custom `dea-notebooks` function called `collect_training_data`, contained within the [dea_tools.classification](../Tools/gen/dea_tools.classification.ipynb) script.  The principal goal of this notebook is to familiarise users with this function so they can extract the appropriate data for their use-case. The default example also highlights extracting a set of useful feature layers for generating a cropland mask forWA.

1. Preview the polygons in our training data by plotting them on a basemap
2. Define a feature layer function to pass to `collect_training_data`
3. Extract training data from the datacube using `collect_training_data`
4. Export the training data to disk for use in subsequent scripts

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages


In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs

import sys
sys.path.insert(1, '../../Tools/')
from dea_tools.bandindices import calculate_indices
from dea_tools.classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

## Analysis parameters

* `path`: The path to the input vector file from which we will extract training data. A default geojson is provided.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**


In [2]:
path = 'data/crop_training_WA.geojson' 
field = 'class'

### Find the number of CPUs

In [3]:
ncpus = round(get_cpu_quota())
print('ncpus = ' + str(ncpus))

ncpus = 2


## Preview input data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. 

> Remember, the class labels **must** be represented by `integers`.


In [4]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,class,geometry
0,1,POINT (116.60407 -31.46883)
1,1,POINT (117.03464 -32.40830)
2,1,POINT (117.30838 -32.33747)
3,1,POINT (116.74607 -31.63750)
4,1,POINT (116.85817 -33.00131)


In [5]:
# Plot training data in an interactive map
input_data.explore(column=field)

## Extracting training data

The function `collect_training_data` takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any `NaN` or `inf` values.

The below variables can be set within the `collect_training_data` function:

* `zonal_stats` : An optional string giving the names of zonal statistics to calculate across each polygon (if the geometries in the vector file are polygons and not points). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.

In addition to the `zonal_stats` parameter, we also need to set up a datacube query dictionary for the Open Data Cube query such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). These options will be added to a query dictionary that will be passed into `collect_training_data` using the parameter `collect_training_data(dc_query=query, ...)`. The query dictionary will be the only argument in the **feature layer function** which we will define and describe in a moment.

> Note: `collect_training_data` also has a number of additional parameters for handling ODC I/O read failures, where polygons that return an excessive number of null values can be resubmitted to the multiprocessing queue. Check out the [docs](../Tools/gen/dea_tools.classification.ipynb) to learn more.


In [6]:
# Set up our inputs to collect_training_data
zonal_stats = None

# Set up the inputs for the ODC query
time = ('2014')
resolution = (-30, 30)
output_crs = 'epsg:3577'

In [7]:
# Generate a new datacube query object
query = {
    'time': time,
    'resolution': resolution,
    'output_crs': output_crs,
}

## Defining feature layers

To create the desired feature layers, we pass instructions to `collect training data` through the `feature_func` parameter. 

* `feature_func`: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The 'feature_func' must accept a 'dc_query' object, and return a single xarray.Dataset or xarray.DataArray containing 2D coordinates (i.e x, y - no time dimension). e.g.

            def feature_function(query):
                dc = datacube.Datacube(app='feature_layers')
                ds = dc.load(**query)
                ds = ds.mean('time')
                return ds

Below, we will define a more complicated feature layer function than the brief example shown above. We will load satellite bands and the ternary Median Abosolute Deviation dataset from the [Landsat 8 geomedian](https://explorer.dea.ga.gov.au/products/ga_ls8c_nbart_gm_cyear_3) product, calculate some additional band indices, and finally append fractional cover percentiles for the photosynthetic vegetation band from the same year: [fc_percentile_albers_annual](https://explorer.dea.ga.gov.au/products/fc_percentile_albers_annual/extents).

In [8]:
def feature_layers(query):
    
    # Connect to the datacube
    dc = datacube.Datacube(app='custom_feature_layers')
    
    # Load ls8 geomedian
    ds = dc.load(product='ga_ls8c_nbart_gm_cyear_3',
                 **query)
    
    # Calculate some band indices
    da = calculate_indices(ds,
                           index=['NDVI', 'LAI', 'MNDWI'],
                           drop=False,
                           collection='ga_gm_3')
    
    # Add Fractional cover percentiles
    fc = dc.load(product='ga_ls_fc_pc_cyear_3',
                 measurements=['pv_pc_10', 'pv_pc_50', 'pv_pc_90'], # only the PV band
                 like=ds.geobox, # will match geomedian extent
                 time=query.get('time'),  # use time if in query
                 dask_chunks=query.get('dask_chunks')  # use dask if in query
                )

    # Merge results into single dataset 
    result = xr.merge([da, fc], compat='override')
    
    return result

Now, we can pass this function to `collect_training_data`. This will take a few minutes to run all 430 samples on the default sandbox as it only has two cpus.

In [ ]:
%%time
column_names, model_input = collect_training_data(
    gdf=input_data,
    dc_query=query,
    ncpus=ncpus,
    return_coords=False,
    field=field,
    zonal_stats=zonal_stats,
    feature_func=feature_layers)

In [10]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

['class', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'sdev', 'edev', 'bcdev', 'count', 'NDVI', 'LAI', 'MNDWI']

[[   1.    989.   1451.   ...    0.27    0.61   -0.5 ]
 [   1.   1233.   1789.   ...    0.23    0.57   -0.39]
 [   1.   1064.   1502.   ...    0.24    0.55   -0.46]
 ...
 [   0.    566.   1009.   ...    0.17    0.25   -0.63]
 [   0.    532.    746.   ...    0.44    0.78   -0.53]
 [   0.    658.   1025.   ...    0.27    0.48   -0.53]]


## Export training data

Once we've collected all the training data we require, we can write the data to disk. This will allow us to import the data in the next step(s) of the workflow.


In [11]:
# Set the name and location of the output file
output_file = "results/test_training_data.txt"

In [18]:
# Export files to disk
np.savetxt(output_file, model_input, header=" ".join(column_names), fmt="%4f")

## Recommended next steps

To continue working through the notebooks in this `Scalable Machine Learning on the ODC` workflow, go to the next notebook `2_Inspect_training_data.ipynb`.

1. **Extracting training data from the ODC (this notebook)**
2. [Inspecting training data](2_Inspect_training_data.ipynb)
3. [Evaluate, optimize, and fit a classifier](3_Evaluate_optimize_fit_classifier.ipynb)
4. [Classifying satellite data](4_Classify_satellite_data.ipynb)
5. [Object-based filtering of pixel classifications](5_Object-based_filtering.ipynb)


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** December 2023

**Compatible datacube version:** 

In [19]:
print(datacube.__version__)

1.8.6


## Tags
<!-- Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex/) -->